In [1]:
import numpy as np
import chemtools
from iodata import load_one
from pyscf import gto, scf
from pyscf.tools import molden, chkfile_util
from numpy.testing import assert_allclose


np.random.seed(123)

# Create and dump inputs/outputs

In [2]:
from iodata import load_one, dump_one

# iodata
mol = load_one('water_neutral.fchk')
dump_one(mol, 'water_iodata.molden')

# pyscf
mol_pyscf = gto.M(
    atom = 'O 0.01086593, 0.00805023, -0.00568808; H 0.52133765,  1.67452412,  0.47604091; H 1.13869228, -0.44555963, -1.34435117',
    basis = 'CC-pVTZ'
)
mf = scf.RHF(mol_pyscf).run()
molden.dump_scf(mf, 'pyscf_molden.molden')

converged SCF energy = -75.6682970590962


In [3]:
mol_pyscf, mo_energy_pyscf, mo_coeff_pyscf, mo_occ_pyscf, irrep_label_pyscf, spins_psycf = molden.load('pyscf_molden.molden')
mol_chemtools, mo_energy_chemtools, mo_coeff_chemtools, mo_occ_chemtools, irrep_label_chemtools, spins_chemtools = molden.load('water_iodata.molden')

Unknown section TITLE


In [4]:
try:
    assert_allclose(mo_energy_pyscf, mo_energy_chemtools)
except AssertionError as e:
    print(e)


Not equal to tolerance rtol=1e-07, atol=0

(shapes (58,), (2, 58) mismatch)
 x: array([-2.065296e+01, -1.201374e+00, -5.081768e-01, -4.519131e-01,
       -4.385704e-01, -1.704449e-02,  1.984404e-02,  3.757741e-01,
        3.877062e-01,  6.822719e-01,  6.849452e-01,  7.491530e-01,...
 y: array([[-19.120402,  -1.010202,  -0.529527,  -0.385675,  -0.309872,
          0.02487 ,   0.097268,   0.368527,   0.416342,   0.491817,
          0.579736,   0.619917,   0.624603,   0.677676,   0.773847,...


In [20]:
try:
    assert_allclose(mo_coeff_chemtools, mo_coeff_pyscf)
except AssertionError as e:
    print(e)


Not equal to tolerance rtol=1e-07, atol=0

(shapes (2, 58, 58), (58, 58) mismatch)
 x: array([[[ 9.732297e-01, -2.049642e-01, -8.922210e-08, ...,
         -9.718058e-08, -7.764849e-07,  1.526411e+00],
        [-9.379156e-03,  4.657901e-01,  2.837934e-07, ...,...
 y: array([[ 9.766285e-01, -4.612257e-03, -3.560010e-16, ..., -4.343873e-01,
        -1.944477e-06,  2.528644e+00],
       [-3.630840e-04,  5.651166e-01, -1.751531e-15, ..., -4.871092e-01,...


In [18]:
try:
    assert_allclose(mo_occ_chemtools[0] + mo_occ_chemtools[1], mo_occ_pyscf)
except AssertionError as e:
    print(e)

In [ ]:
!cat water_neutral.fchk

# Try to load from regular fchk 

In [ ]:
mol_pyscf = gto.Mole()
mol_pyscf.atom = 'water_neutral.fchk'
mol_pyscf.build()

# Try to load from molden

In [ ]:
# first line must be deleted, otherwise it the format is incorrect 
molden_file = 'water.molden'

def delete_first_line(text_file):
    with open(text_file, 'r+') as f:
        to_read = f.read().splitlines(True)
        to_write = ' '.join([i for i in to_read[1:]])
        f.seek(0)
        f.write(to_write)
        f.truncate()

delete_first_line('water.molden')

mol_pyscf = gto.Mole()
mol_pyscf.atom = molden_file
mol_pyscf.build()

# Load manually from chemtools

In [ ]:
mol = chemtools.Molecule.from_file('water_neutral.fchk')

def make_pyscf_input(numbers, coords):
    numbers = np.around(numbers, decimals=1)
    return list(zip(numbers, coords))

mol_pyscf = gto.M(
    atom = make_pyscf_input(mol.numbers, mol.coordinates),
    basis = mol.obasis_name
)
mf = scf.RHF(mol_pyscf).run()

grid_3d = np.random.random((1,3))
# test a calculation
mol_pyscf.eval_gto('GTOval_sph', grid_3d)

# Load using pyscf.tools module and test some calculations

In [ ]:
from numpy.testing import assert_allclose

In [ ]:
mol_pyscf, mo_energy_pyscf, mo_coeff_pyscf, mo_occ_pyscf, irrep_label_pyscf, spins_pyscf = molden.load('water.molden')
mol_chemtools = chemtools.Molecule.from_file('water_neutral.fchk')

In [ ]:
mo_coeff_pyscf[0], mol_chemtools.mo.coefficient[0]